# Introduction to Pandas

In this tutorial, we will learn how to use Pandas by analyzing a real-world dataset.

The dataset that we are going to analyze is the TED talk dataset which is available on Kaggle (https://www.kaggle.com/datasets/ahmadfatani/ted-talks-dataset). The dataset contains information about all video recordings of TED Talks uploaded to the official TED.com website until April 18th, 2020. It contains information about all talks including the number of views, tags, posted-date, speakers and titles.

Note that you do not have to download the dataset from Kaggle since the data is already contained in the Github repository.

In [1]:
import pandas as pd

ted_df = pd.read_csv('../ted_talk_dataset/ted_main.csv')

## What where the "best" TED events to attend?

**Caution:** We are interested in the *event* and NOT in individual *talks*.

We first have to define what we consider "best". There is a large variety of options, for example ...

- The event that had the most talks (Idea: If there are a lot of talks, it is worth going there)
- Look at the ratings data to see whether we can judge individual talks based on the ratings and aggregate individual ratings to rate the event.
- Simply sum up the number of views for all talks at each event. However, this could mean that there is one "good" talk and many "bad" ones + the number is influenced by the number of talks.
- Simply take the average (or median?) of the number of views of all talks at each event.
- If we are judging based on views, it might be good to normalize the data as the number of views might increase over time (this is something that should be verified)

### Which event had the most talks?

In [2]:
ted_df.event.value_counts().sort_values()

event
TEDxZurich       1
TEDxSFU          1
TEDxPortland     1
TEDxHamburg      1
TEDxCrenshaw     1
                ..
TED2015         75
TED2013         77
TED2016         77
TED2009         83
TED2014         84
Name: count, Length: 355, dtype: int64

TED2014 had the most talks. So if we consider the best TED event to be the one with the most talks, it's the event "TED2014".

Furthermore, note that there are many events that have a low number of talks (e.g., only one talk).

### Which event had the most views in total?

In [3]:
ted_df.groupby('event').views.sum().sort_values().tail(10)

event
TED2011           137750504
TED2016           139571905
TED2012           144497608
TEDGlobal 2012    145070549
TED2009           145656385
TED2006           147345533
TED2015           150826305
TEDGlobal 2013    170554736
TED2014           174121423
TED2013           177307937
Name: views, dtype: int64

If we compare this dataframe to the previous one, we can see the events with the most total views appear among the talks that have the most talks (*). There seems to be a correlation between those "variables". Note that Pandas allows to quantify the correlation between different columns with the `corr()` function.

(*) This seems plausible since the more talks an event has, the easier it is to collect many views.

In [4]:
most_talks_series = ted_df.event.value_counts().sort_values()
most_views_series = ted_df.groupby('event').views.sum().sort_values()

# We can now combine the two series based on the index
best_df = pd.DataFrame({
    'most_talks': most_talks_series,
    'most_views': most_views_series
})


# Computes the pairwise correlation of columns
print(best_df.corr())

            most_talks  most_views
most_talks    1.000000    0.961884
most_views    0.961884    1.000000


Well, obviously, both rows are extremely well correlated ...

In [5]:
best_df.sort_values('most_talks').tail(5)

,most_talks,most_views
event,,
TED2015,75,150826305
TED2016,77,139571905
TED2013,77,177307937
TED2009,83,145656385
TED2014,84,174121423


### Which event had the most views on average?

In [6]:
ted_df.groupby('event').views.mean().sort_values().tail(5)

event
TEDxNorrkoping        6569493.0
TEDxCreativeCoast     8444981.0
TEDxBloomington       9484259.5
TEDxHouston          16140250.5
TEDxPuget Sound      34309432.0
Name: views, dtype: float64

Ok, that's interesting. If we consider the mean instead of the sum, a totally different set of events turns out best. But wait ... Haven't we mentioned that there are some events that have only one talk? Maybe that has some effect on the result.

Let's print the different statistics all together. This can be easily done with the `agg()` method.

In [7]:
ted_df.groupby('event').views.agg(['mean', 'count', 'sum']).sort_values('mean').tail(5)

,mean,count,sum
event,,,
TEDxNorrkoping,6569493.0,1,6569493
TEDxCreativeCoast,8444981.0,1,8444981
TEDxBloomington,9484259.5,2,18968519
TEDxHouston,16140250.5,2,32280501
TEDxPuget Sound,34309432.0,1,34309432


Indeed, the events with high mean values only have just 1 or 2 events. So, this is definitely something we should have in mind when we choose our final criterion for the best number of views.

In [8]:
ted_df.groupby('event').views.agg(['mean', 'count', 'sum']).sort_values('sum').tail(5)

,mean,count,sum
event,,,
TED2006,3.274345e+06,45,147345533
TED2015,2.011017e+06,75,150826305
TEDGlobal 2013,2.584163e+06,66,170554736
TED2014,2.072874e+06,84,174121423
TED2013,2.302700e+06,77,177307937


### Remark

Note that `agg()` can also take custom functions!

In [9]:
def custom_agg_func(series):
    # Returns always 1
    return 1

ted_df.groupby('event').views.agg([custom_agg_func])

,custom_agg_func
event,
AORN Congress,1
Arbejdsglaede Live,1
BBC TV,1
Bowery Poetry Club,1
Business Innovation Factory,1
...,...
The Do Lectures,1
Toronto Youth Corps,1
University of California,1


In [10]:
ted_df.groupby('event').views.apply(custom_agg_func)

event
AORN Congress                  1
Arbejdsglaede Live             1
BBC TV                         1
Bowery Poetry Club             1
Business Innovation Factory    1
                              ..
The Do Lectures                1
Toronto Youth Corps            1
University of California       1
Web 2.0 Expo 2008              1
World Science Festival         1
Name: views, Length: 355, dtype: int64